In [43]:
from itertools import combinations

import numpy as np
import scipy as sp
from skbio.stats.distance import DistanceMatrix
from skbio.util import get_data_path
import biom
import pandas as pd
from qiime2 import Visualization

def _validate_parameters(dm, num_prototypes, seedset=None):
    '''Validate the paramters for each algorithm.
    Parameters
    ----------
    dm: skbio.stats.distance.DistanceMatrix
        Pairwise distances for all elements in the full set S.
    num_prototypes: int
        Number of prototypes to select for distance matrix.
        Must be >= 2, since a single prototype is useless.
        Must be smaller than the number of elements in the distance matrix,
        otherwise no reduction is necessary.
    seedset: iterable of str
        A set of element IDs that are pre-selected as prototypes. Remaining
        prototypes are then recruited with the prototype selection algorithm.
        Warning: It will most likely violate the global objective function.
    Raises
    ------
    ValueError
        The number of prototypes to be found should be at least 2 and at most
        one element smaller than elements in the distance matrix. Otherwise, a
        ValueError is raised.
        The IDs in the seed set must be unique, and must be present in the
        distance matrix. Otherwise, a ValueError is raised.
        The size of the seed set must be smaller than the number of prototypes
        to be found. Otherwise, a ValueError is raised.
    '''
    if num_prototypes < 2:
        raise ValueError("'num_prototypes' must be >= 2, since a single "
                         "prototype is useless.")
    if num_prototypes >= dm.shape[0]:
        raise ValueError("'num_prototypes' must be smaller than the number of "
                         "elements in the distance matrix, otherwise no "
                         "reduction is necessary.")
    if seedset is not None:
        seeds = set(seedset)
        if len(seeds) < len(seedset):
            raise ValueError("There are duplicated IDs in 'seedset'.")
        if not seeds < set(dm.ids):  # test if set A is a subset of set B
            raise ValueError("'seedset' is not a subset of the element IDs in "
                             "the distance matrix.")
        if len(seeds) >= num_prototypes:
            raise ValueError("Size of 'seedset' must be smaller than the "
                             "number of prototypes to select.")

def prototype_selection_constructive_maxdist(dm, num_prototypes, seedset=None):
    '''Heuristically select k prototypes for given distance matrix.
       Prototype selection is NP-hard. This is an implementation of a greedy
       correctness heuristic: Greedily grow the set of prototypes by adding the
       element with the largest sum of distances to the non-prototype elements.
       Start with the two elements that are globally most distant from each
       other. The set of prototypes is then constructively grown by adding the
       element showing largest sum of distances to all non-prototype elements
       in the distance matrix in each iteration.
    Parameters
    ----------
    dm: skbio.stats.distance.DistanceMatrix
        Pairwise distances for all elements in the full set S.
    num_prototypes: int
        Number of prototypes to select for distance matrix.
        Must be >= 2, since a single prototype is useless.
        Must be smaller than the number of elements in the distance matrix,
        otherwise no reduction is necessary.
    seedset: iterable of str
        A set of element IDs that are pre-selected as prototypes. Remaining
        prototypes are then recruited with the prototype selection algorithm.
        Warning: It will most likely violate the global objective function.
    Returns
    -------
    list of str
        A sequence holding selected prototypes, i.e. a sub-set of the
        IDs of the elements in the distance matrix.
    Raises
    ------
    ValueError
        The number of prototypes to be found should be at least 2 and at most
        one element smaller than elements in the distance matrix. Otherwise, a
        ValueError is raised.
    Notes
    -----
    Timing: %timeit -n 100 prototype_selection_constructive_maxdist(dm, 100)
            100 loops, best of 3: 1.43 s per loop
            where the dm holds 27,398 elements
    function signature with type annotation for future use with python >= 3.5:
    def prototype_selection_constructive_maxdist(dm: DistanceMatrix,
    num_prototypes: int, seedset: List[str]) -> List[str]:
    '''
    _validate_parameters(dm, num_prototypes, seedset)

    # initially mark all elements as uncovered, i.e. as not being a prototype
    uncovered = np.asarray([np.True_] * dm.shape[0])
    res_set, num_found_prototypes = [], 0

    if seedset is not None:
        # mark elements in the seedset as found
        seedset = set(seedset)
        for idx, id_ in enumerate(dm.ids):
            if id_ in seedset:
                uncovered[idx] = np.False_
                res_set.append(idx)
    else:
        # the first two prototypes are those elements that have the globally
        # maximal distance in the distance matrix. Mark those two elements as
        # being covered, i.e. prototypes
        res_set = list(np.unravel_index(dm.data.argmax(), dm.data.shape))
        uncovered[res_set] = np.False_

    # counts the number of already found prototypes
    num_found_prototypes = len(res_set)

    # repeat until enough prototypes have been selected:
    # the new prototype is the element that has maximal distance sum to all
    # non-prototype elements in the distance matrix.
    while num_found_prototypes < num_prototypes:
        max_elm_idx = (dm.data[res_set, :].sum(axis=0) * uncovered).argmax()
        uncovered[max_elm_idx] = np.False_
        num_found_prototypes += 1
        res_set.append(max_elm_idx)

    # return the ids of the selected prototype elements
    return [dm.ids[idx] for idx, x in enumerate(uncovered) if not x]


In [2]:
#Wang2020ASD
dm = DistanceMatrix.read(get_data_path('/mnt/home/djin/ceph/snakemake/data/Wang2020/beta/data/distance-matrix.tsv'))
prototype_selection_constructive_maxdist(dm, 30, seedset=None)

['ERR2608608',
 'ERR2608657',
 'ERR2608613',
 'ERR2608671',
 'ERR2608662',
 'ERR2608655',
 'ERR2608663',
 'ERR2608620',
 'ERR2608609',
 'ERR2608665',
 'ERR2608628',
 'ERR2608632',
 'ERR2608672',
 'ERR2608658',
 'ERR2608659',
 'ERR2608637',
 'ERR2608634',
 'ERR2608984',
 'ERR2608656',
 'ERR2608986',
 'ERR2608646',
 'ERR2608645',
 'ERR2608635',
 'ERR2608661',
 'ERR2608644',
 'ERR2608648',
 'ERR2608618',
 'ERR2608611',
 'ERR2608615',
 'ERR2608654']

In [6]:
#Qin2012T2D
!pwd

/mnt/ceph/users/djin/jupyter_notebook/Meta_diseases_analyses/ipynb


In [29]:
#Filter the biom table to include 170 matched samples only:
table = biom.load_table("/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D.biom")
metadata = pd.read_csv("/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_metadata_new_matched_170.txt",sep = '\t', index_col = 0)
features_to_keep = metadata.index.tolist()
table_filt = table.filter(features_to_keep, axis="sample")
with biom.util.biom_open('/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype.biom', 'w') as f:
    table_filt.to_hdf5(f, 'counts')

In [8]:
#Qin2012T2D_170samples.qza
qiime tools import \
 --input-path /mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype.biom \
 --type 'FeatureTable[Frequency]' \
 --input-format BIOMV210Format \
 --output-path /mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype.qza

qiime diversity beta \
 --i-table /mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype.qza \
 --p-metric 'braycurtis' \
 --o-distance-matrix /mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype_beta.qza

In [30]:
dm = DistanceMatrix.read(get_data_path('/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype/data/distance-matrix.tsv'))
features_to_keep = prototype_selection_constructive_maxdist(dm, 60, seedset=None)

In [32]:
#features_to_keep

In [33]:
#Filter the biom table:
table = biom.load_table("/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D.biom")
table_filt = table.filter(features_to_keep, axis="sample")
with biom.util.biom_open('/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototype.biom', 'w') as f:
    table_filt.to_hdf5(f, 'counts')

In [34]:
#filter the metadata
Qin2010T2D = pd.read_csv("/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_metadata_new_matched_170.txt", sep = '\t')

In [35]:
#filter Qin2010T2D
Qin2010T2D_prototype = Qin2010T2D.loc[Qin2010T2D['featureid'].isin(features_to_keep)]

In [36]:
Qin2010T2D_prototype['Diabetic'] == 'Y'

0       True
2       True
4       True
7       True
11      True
12      True
16      True
17      True
19      True
26     False
36     False
38     False
39     False
41     False
43     False
49     False
51     False
52     False
59     False
61      True
62      True
66      True
67      True
71      True
72      True
74      True
76      True
77      True
78      True
80      True
91     False
92     False
99     False
100    False
109    False
110    False
121    False
125    False
127    False
129    False
130    False
131    False
132    False
133    False
134     True
136     True
140     True
144     True
146     True
148     True
152     True
153     True
155     True
156     True
158     True
160     True
162     True
163     True
167     True
168     True
Name: Diabetic, dtype: bool

In [37]:
Qin2010T2D_prototype.to_csv('/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/Qin2012T2D_prototye.txt',index=False, sep = "\t")

In [ ]:
#build the classifier
#!/bin/sh
#SBATCH --job-name=q2
#SBATCH --mail-type=END
#SBATCH --mail-user=dj2080@nyu.edu
#SBATCH --output=slurm_%j.out


qiime tools import \
 --input-path ../Qin2012T2D_prototype.biom \
 --type 'FeatureTable[Frequency]' \
 --input-format BIOMV210Format \
 --output-path Qin2012T2D_prototype.qza


qiime sample-classifier classify-samples \
  --i-table Qin2012T2D_prototype.qza \
  --m-metadata-file ../Qin2012T2D_prototye.txt \
  --m-metadata-column Diabetic \
  --p-optimize-feature-selection \
  --p-parameter-tuning \
  --p-estimator GradientBoostingClassifier \
  --p-n-estimators 20 \
  --p-random-state 123 \
  --output-dir moving-pictures-classifier_GB_prototype

In [42]:
Visualization.load('/mnt/home/djin/ceph/snakemake/data/Qin2012T2D/classify/moving-pictures-classifier_GB_prototype/accuracy_results.qzv')

<visualization: Visualization uuid: 0b1cd748-3268-4f36-9376-ba8b5be4a778>

In [44]:
table = biom.load_table("/mnt/home/djin/ceph/snakemake/data/Dan2020ASD_rl150/classify/phylo-table-copy/data/feature-table.biom")


In [48]:
table

6982 x 60 <class 'biom.table.Table'> with 337494 nonzero entries (80% dense)

In [51]:
table.to_dataframe().index

Index(['GUT_GENOME113464', 'GUT_GENOME100501', 'GUT_GENOME258656',
       'GUT_GENOME265996', 'n4', 'n5', 'GUT_GENOME252369', 'GUT_GENOME102756',
       'n8', 'n9',
       ...
       'GUT_GENOME139700', 'GUT_GENOME134384', 'n6974', 'n6975', 'n6976',
       'n6977', 'GUT_GENOME275558', 'n6979', 'GUT_GENOME258988', 'n6981'],
      dtype='object', length=6982)